In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import re
import scipy.spatial.distance as sp

In [4]:
!python3 --version

Python 3.10.8


In [121]:
df = pd.read_pickle("userTimeLines_100722_FinalTweets.pkl")

In [122]:

#1. remove mentions
df['clean_text'] = df.text.apply(lambda tweet: re.sub("@[A-Za-z0-9_]+","", tweet))

#2. remove urls
df['clean_text'] = df.clean_text.apply(lambda tweet: re.sub(r"http\S+", "", tweet))

#3. remove double space between words 
df['clean_text'] = df.clean_text.apply(lambda tweet: re.sub(" +", " ", tweet))

df['clean_text'] = df.clean_text.apply(lambda tweet: re.sub(r'[^\w\s]', " ",tweet))



In [5]:

# read glove vectors
term2vec = {}
IN = open ("/Users/eotenen/Downloads/glove.6B.50d.txt", "r")

d = IN.readlines()
IN.close()


In [9]:
for line in d:
        l = line.strip('\n')
        vals = l.split(" ")
        term2vec[vals[0]] = np.asfarray(vals[1:])

sentences = ["Gas prices have decreased to their pre-Russian invasion of Ukraine levels",
             "Wisconsin's archaic abortion ban is older than 20 states",
             "The USA has a gun homicide rate 26 times higher than our peers.",
             "86% of Americans and 82% of gun owners support requiring all gun buyers to pass a background check",
             "Joe Biden’s $36 billion for a union pension fund is the largest private pension bailout in American history",
             "Biden says he released 22 years of my tax returns but Donald Trump hasn’t released a single one",
             "When the New York State Senate voted to legalize abortion in 1970, 12 Republican senators voted in favor of it"]

for sentence in sentences:
    words = sentence.split(" ")
    v = [term2vec[word] for word in words]
    vd = [sp.euclidean(v1,v2) for v1, v2 in zip(v,v[1:])]
    print(sentence, np.std(vd))


KeyError: 'Gas'

In [119]:
sentences = [*df.clean_text]
ids = [*df.id_str]
sentences

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',


In [115]:
for line in d:
    l = line.strip('\n')
    vals = l.split(" ")
    term2vec[vals[0]] = np.asfarray(vals[1:])

    
for sentence in sentences:
    words = sentence.split(" ")
    v = [term2vec[word] for word in words]
    vd = [sp.euclidean(v1,v2) for v1, v2 in zip(v,v[1:])]
    print(sentence, np.std(vd))


KeyError: ''

In [ ]:
for idx, row in df.iterrows():
    for paragraph in (row['clean_text'].split(' ')):
        v = (term2vec[word] for word in paragraph)
        print(v)